In [ ]:
import asyncio

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, RemoveMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableGenerator, RunnableMap

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

## Define a prompt -> LLM -> Output Parser

In [ ]:
prompt_template = PromptTemplate.from_template(
    template="""You are a {role} assistant.

Question: {question}
Answer:"""
)

prompt_value = prompt_template.invoke(
    {"role": "bad", "question": "What is the capital of France?"}
)

prompt_value

In [ ]:
chat_prompt_tpl = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        template="""You are a {role} assistant. You need to extract the first name and last name of the person from the question.
Format into a JSON object with the keys 'first_name' and 'last_name'."""
    ),
    HumanMessage(content="Hi my name is John Doe")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Step 1: Generate the prompt
prompt_value = chat_prompt_tpl.invoke(
    {
        "role": "bad",
        "question": "What is the capital of France?",
        "messages": [AIMessage(content="I don't know")],
    }
)

# Step 2: Generate the LLM call
llm_call = await llm.ainvoke(prompt_value)
print(llm_call)

# Step 3: Parse the LLM call
parser = JsonOutputParser()
parsed_llm_call = parser.invoke(llm_call)
parsed_llm_call["first_name"], parsed_llm_call["last_name"]

## Runnable Syntax Suger

In [ ]:
chat_prompt_tpl = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            template="""You are a {role} assistant. You need to extract the first name and last name of the person from the question.
Format into a JSON object with the keys 'first_name' and 'last_name'."""
        ),
        HumanMessage(content="Hi my name is John Doe"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

parser = JsonOutputParser()

"""
chain = RunnableSequence([
    chat_prompt_tpl,
    llm,
    parser
])
"""

# operator override
chain = chat_prompt_tpl | llm | parser

"""
def pipe(left, right): ...

chain = pipe(pipe(chat_prompt_tpl, llm), parser)
"""

result = chain.invoke({"role": "bad", "question": "What is the capital of France?"})
result

## Batching

In [ ]:
results = chain.batch([
    {"role": "bad", "question": "What is the capital of France?"},
    {"role": "helpful", "question": "What is the capital of Germany?"},
    {"role": "good", "question": "What is the capital of Italy?"},
])

results

In [ ]:
from langchain_core.messages import AIMessageChunk


def truncate_text(text: AIMessage):
    return text.content[:100]

# RunnableGenerator
async def truncate_text_stream(iter):
    total_length = 0
    async for chunk in iter:
        if total_length > 200:
            break

        total_length += len(chunk.content)
        yield chunk.content

    yield "END OF TEXT"

# async for chunk in llm.astream("Hi, tell me a long story"):
#     print(chunk.content, end="")

def capitalize(text: AIMessage):
    return text.content.upper()

async def capitalize_stream(iter):
    async for chunk in iter:
        yield chunk.content.upper()

chain_1 = llm | RunnableGenerator(truncate_text_stream)
async for chunk in chain_1.astream("Hi, tell me a long story"):
    print(chunk, end="")

In [ ]:
from operator import itemgetter

with open("doc.txt", "r") as f:
    context = f.read()

prompt = PromptTemplate.from_template(
    template="""
    You are a helpful assistant.
    Context: {context}
Extract all the useful API names from the context. Format in a JSON key value pairs, e.g. {{ "Runnable": "A class that does xyz", "RunnableLambda": "A class that does xyz" }}
        """
)

chain_1 = prompt | llm | JsonOutputParser()
chain_2 = prompt | llm | StrOutputParser()

chain = RunnableMap(
    {
        "structured": chain_1 | itemgetter("Messages"),
        "unstructured": chain_2 | (lambda x: x[:100]),
    }
)

# {
#     "structured": "explaination of prompt template",
#     "unstructured": "the first 100 characters of the unstructured output"
# }

# r = chain.invoke({"context": context})

# r['structured'] = r['structured']['PromptTemplate']
# r['unstructured'] = r['unstructured'][:100]

# chain.get_graph().print_ascii()

In [ ]:
!pip install grandalf

In [ ]:
from pydantic import BaseModel

class ApiItem(BaseModel):
    name: str
    description: str

class ApiList(BaseModel):
    items: list[ApiItem]
    name: str

prompt = PromptTemplate.from_template(
    template="""
    You are a helpful assistant.
    Context: {context}
    Extract all the useful API names from the context.
    """
)

llm = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(
    ApiList
)

async for chunk in llm.astream({ "context": context }):
    print(chunk)


"""
{
    "items": [
        {
            "name": "Runnable",
            "description": "A class that does xyz"
        }
    ]
    "name
}

items=[ApiItem(name='Installation')]
"""

In [18]:
code = """
class User {
  firstName: string = "";
  lastName: string = "";
  username: string = "";
}

export default User;
"""

refactor_prompt = """
Replace the "username" property with an "email" property. Respond only
with code, and with no markdown formatting.
"""


class CodeRefactor(BaseModel):
    refactored_code: str

prompt = ChatPromptTemplate.from_messages([
    ("system", refactor_prompt),
    ("user", "{code}"),
])

llm = ChatOpenAI(model="gpt-4o", temperature=0).bind(
    prediction={
        "type": "content",
    },
)

chain = prompt | llm

async for chunk in chain.astream({"code": code}):
    print(chunk.content, end="")

class User {
  firstName: string = "";
  lastName: string = "";
  email: string = "";
}

export default User;

In [59]:
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver
from dataclasses import dataclass

@dataclass
class Context:
    code: str

@task
def hello() -> str:
    return "hello"

@task
async def refactor(code: str) -> str:
    return await hello()


@entrypoint(checkpointer=None)
async def agent(code: Context) -> str:
    await refactor(code.code)
    await refactor(code.code)
    return ""

In [60]:
async for chunk in agent.astream(Context("code")):
    print(chunk)

{'hello': 'hello'}
{'refactor': 'hello'}
{'hello': 'hello'}
{'refactor': 'hello'}
{'agent': ''}


In [1]:
from typing import List, Optional

from pydantic import BaseModel


class OutputFormat(BaseModel):
    preference: str
    sentence_preference_revealed: str


class TelegramPreferences(BaseModel):
    preferred_encoding: Optional[List[OutputFormat]] = None
    favorite_telegram_operators: Optional[List[OutputFormat]] = None
    preferred_telegram_paper: Optional[List[OutputFormat]] = None


class MorseCode(BaseModel):
    preferred_key_type: Optional[List[OutputFormat]] = None
    favorite_morse_abbreviations: Optional[List[OutputFormat]] = None


class Semaphore(BaseModel):
    preferred_flag_color: Optional[List[OutputFormat]] = None
    semaphore_skill_level: Optional[List[OutputFormat]] = None


class TrustFallPreferences(BaseModel):
    preferred_fall_height: Optional[List[OutputFormat]] = None
    trust_level: Optional[List[OutputFormat]] = None
    preferred_catching_technique: Optional[List[OutputFormat]] = None


class CommunicationPreferences(BaseModel):
    telegram: TelegramPreferences
    morse_code: MorseCode
    semaphore: Semaphore


class UserPreferences(BaseModel):
    communication_preferences: CommunicationPreferences
    trust_fall_preferences: TrustFallPreferences


class TelegramAndTrustFallPreferences(BaseModel):
    pertinent_user_preferences: UserPreferences

In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
bound = llm.with_structured_output(TelegramAndTrustFallPreferences)

conversation = """Operator: How may I assist with your telegram, sir?
Customer: I need to send a message about our trust fall exercise.
Operator: Certainly. Morse code or standard encoding?
Customer: Morse, please. I love using a straight key.
Operator: Excellent. What's your message?
Customer: Tell him I'm ready for a higher fall, and I prefer the diamond formation for catching.
Operator: Done. Shall I use our "Daredevil" paper for this daring message?
Customer: Perfect! Send it by your fastest carrier pigeon.
Operator: It'll be there within the hour, sir."""

r = bound.invoke(
    f"""Extract the preferences from the following conversation:
<convo>
{conversation}
</convo>"""
)

In [39]:
import json

# print(json.dumps(r, indent=2))
r.dict()

/var/folders/w2/73qnm2kd2437hd9g4sswpjkc0000gq/T/ipykernel_92444/1694558619.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  r.dict()


{'pertinent_user_preferences': {'communication_preferences': {'telegram': {'preferred_encoding': [{'preference': 'morse code',
      'sentence_preference_revealed': 'I need to send a message about our trust fall exercise.'}],
    'favorite_telegram_operators': [{'preference': 'straight key',
      'sentence_preference_revealed': 'I love using a straight key.'}],
    'preferred_telegram_paper': [{'preference': 'Daredevil',
      'sentence_preference_revealed': "Shall I use our 'Daredevil' paper for this daring message?"}]},
   'morse_code': {'preferred_key_type': [{'preference': 'straight key',
      'sentence_preference_revealed': 'I love using a straight key.'}],
    'favorite_morse_abbreviations': None},
   'semaphore': {'preferred_flag_color': None, 'semaphore_skill_level': None}},
  'trust_fall_preferences': {'preferred_fall_height': [{'preference': 'higher fall',
     'sentence_preference_revealed': "Tell him I'm ready for a higher fall."}],
   'trust_level': None,
   'preferred_c